In [1]:
import json
import geopandas as gpd
import pandas as pd
import WGS1984
import requests
import numpy as np
from math import radians, cos, sin, asin, sqrt

In [2]:
def haversine(lon1, lat1, lon2, lat2): 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 
    return c * r * 1000

In [3]:
def get_G(x):#距离衰减计算公式
    if x <= 1098:
        G = (np.exp(-0.5 * (x/1098)**2) - np.exp(-0.5)) / (1-np.exp(-0.5))
        return G
    else:
        return 0

def get_Rj(x):#$R_{j}$ 计算
    x = x.reset_index()
    sj = x['人数'][0]
    dt = 0
    for i in range(len(x)):
        vl = x['pop'][i] * get_G(x['length'][i])
        dt += vl
    return sj/dt

def get_A(x):#$A_{i}$ 计算
    x = x.reset_index()
    dt = 0
    for i in range(len(x)):
        vl = x['vl_R'][i] * get_G(x['length'][i])
        dt += vl
    return dt

In [16]:
net = gpd.read_file("/Users/creative/Documents/python/Accessibility/2SFCA/pop_net.geojson")
school = gpd.read_file("/Users/creative/Documents/python/Accessibility/2SFCA/深圳学校.geojson")
hospital = gpd.read_file("/Users/creative/Documents/python/Accessibility/2SFCA/社康面积.geojson")

In [5]:
#转换坐标系 转换为可计算的投影坐标系
school = school.to_crs(epsg=3857)
hospital = hospital.to_crs(epsg=3857)

In [9]:
def step01(supply_df,demand_df):
    #第一步计算
    #对学校进行缓冲
    supply_copy = supply_df.copy() 
    supply_copy['geometry'] = supply_copy['geometry'].buffer(1098) 
    supply_copy = gpd.sjoin(supply_copy,demand_df,op='contains') #建立空间连接，将在缓冲区内的小区连接到公园中,求得公园与小区之间的直线距离
    supply_copy['length'] = supply_copy[['x_left','y_left','x_right','y_right']].apply(lambda x:haversine(x[0],x[1],x[2],x[3]),axis=1)
    supply_vk = supply_copy.groupby(by='id_left').apply(get_Rj).reset_index()
    supply_vk.columns=['id','vl_R']
    supply_concat = pd.merge(supply_df,supply_vk,on='id')
    return supply_concat

In [10]:
def step02(supply_concat,demand):
    #第二步计算
    #对住宅区进行缓冲
    demand_copy = demand.copy() 
    demand_copy['geometry'] = demand_copy['geometry'].buffer(1098)
    demand_copy = gpd.sjoin(demand_copy,supply_concat,op='intersects')
    demand_copy['length'] = demand_copy[['x_right', 'y_right','x_left', 'y_left']].apply(lambda x:haversine(x[0],x[1],x[2],x[3]),axis=1)
    demand_vl = demand_copy.groupby(by='id_left').apply(get_A).reset_index()

    demand_vl.columns=['id','vl_R']
    dt = demand.merge(demand_vl,on='id')
    return dt

In [13]:
dt = step02(step01(school,net),net)

In [70]:
dt.to_file('/Users/creative/Desktop/研究边界.shp',encoding = 'gb18030') 

In [18]:
hospital["id"] = list(range(0,len(hospital)))
hospital

,机构名称,医生,病床,级别,地址,行政区划,机构分类管理代码,卫生机构类别代码,lng,lat,总人数,服务人数,area,x,y,geometry,id
0,龙田社区健康服务中心,35,None,二类社康中心,深圳市坪山区坑梓街道龙田社区新屋路3号,坪山区,None,二类社康中心,114.355203,22.758797,58378.0,35,400,114.355203,22.758797,POINT (114.35520 22.75880),0
1,白石龙社康中心,56,None,二类社康中心,龙泽苑B栋裙楼一、二层,龙华区,None,二类社康中心,114.038572,22.604006,93232.0,56,400,114.038572,22.604006,POINT (114.03857 22.60401),1
2,玖龙玺社区健康服务中心,56,None,一类社康中心,深圳市龙华区民治街道龙光玖龙玺花园住宅B栋西角1-2层,龙华区,None,一类社康中心,114.035383,22.602450,93232.0,56,1000,114.035383,22.602450,POINT (114.03538 22.60245),2
3,深圳市龙华区人民医院星河传奇社区健康服务中心,56,None,二类社康中心,深圳市龙华区民治街道星河传奇3期商厦1栋B座1-2层,龙华区,None,二类社康中心,114.014494,22.633275,93232.0,56,400,114.014494,22.633275,POINT (114.01449 22.63328),3
4,深圳市龙华区人民医院新彩苑社区健康服务中心,56,None,二类社康中心,民治街道新区大道白石龙一区新龙大厦1-2层,龙华区,None,二类社康中心,114.031291,22.605910,93232.0,56,400,114.031291,22.605910,POINT (114.03129 22.60591),4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
700,深圳市福田区学苑社区健康服务站,2,None,社康站,香蜜湖路3008号学苑大厦,福田区,None,社康站,113.894435,22.520654,4000.0,2,150,113.894435,22.520654,POINT (113.89444 22.52065),700
701,深圳市福田区渔农社区健康服务中心,2,None,二类社康中心,福田区渔农村明津广场商业1楼110-01号铺,福田区,None,二类社康中心,114.067572,22.518268,4000.0,2,400,114.067572,22.518268,POINT (114.06757 22.51827),701
702,南山区医疗集团总部孖洲岛社区健康服务中心,2,None,社康站,广东省深圳市南山区孖洲岛边检消防大楼首层,南山区,None,社康站,113.843081,22.499506,3886.0,2,150,113.843081,22.499506,POINT (113.84308 22.49951),702
703,深圳市南山区医疗集团总部海湾社区健康服务中心,2,None,二类社康中心,广东省深圳市南山区蛇口招东路7号铺,南山区,None,二类社康中心,113.880782,22.513324,337.0,0,400,113.880782,22.513324,POINT (113.88078 22.51332),703
